In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, Point
import ast
import rioxarray as rxr
from pyproj import Proj, Transformer, CRS
from tqdm import tqdm

In [2]:
def get_building_footprint_area_data(ground_df):
    buildings_df = pd.read_csv('Processed_Building_Footprints.csv')
    buildings_df = buildings_df.drop(columns = ['path', 'id', 'tessellate', 'extrude', 'visibility'])

    buildings_df['coordinates'] = buildings_df['coordinates'].apply(ast.literal_eval)
    buildings_df['geometry'] = buildings_df['coordinates'].apply(Polygon)

    gdf_buildings = gpd.GeoDataFrame(buildings_df, geometry='geometry', crs="EPSG:4326")

    gdf_ground = gpd.GeoDataFrame(
        ground_df,
        geometry=gpd.points_from_xy(ground_df['Longitude'], ground_df['Latitude']),
        crs="EPSG:4326"
    )


    gdf_buildings = gdf_buildings.to_crs("EPSG:32618")
    gdf_ground = gdf_ground.to_crs("EPSG:32618")

    def calculate_building_area(point, buildings, buffer_size):
        buffer = point.buffer(buffer_size)
        intersecting_buildings = buildings[buildings.intersects(buffer)]
        total_area = intersecting_buildings.geometry.intersection(buffer).area.sum()
        return total_area

    gdf_ground['building_area_25']  = gdf_ground.geometry.apply(lambda pt: calculate_building_area(pt, gdf_buildings, 25))
    gdf_ground['building_area_50']  = gdf_ground.geometry.apply(lambda pt: calculate_building_area(pt, gdf_buildings, 50))
    gdf_ground['building_area_100'] = gdf_ground.geometry.apply(lambda pt: calculate_building_area(pt, gdf_buildings, 100))

    # Optionally, convert back to a pandas DataFrame if you don't need the geometry column:
    result_df = pd.DataFrame(gdf_ground.drop(columns='geometry'))

    return(result_df)

In [3]:
def get_bands(df):
    
    # Load the GeoTIFF data
    data = rxr.open_rasterio("S2_sample.tiff")
    tiff_crs = data.rio.crs

    # Read the Excel file using pandas
    latitudes = df['Latitude'].values
    longitudes = df['Longitude'].values

    # 3. Convert lat/long to the GeoTIFF's CRS
    # Create a Proj object for EPSG:4326 (WGS84 - lat/long) and the GeoTIFF's CRS
    proj_wgs84 = Proj(init='epsg:4326')  # EPSG:4326 is the common lat/long CRS
    proj_tiff = Proj(tiff_crs)
    
    # Create a transformer object
    transformer = Transformer.from_proj(proj_wgs84, proj_tiff)

    B01_values = []
    B02_values = []
    B03_values = []
    B04_values = []
    B05_values = []
    B06_values = []
    B07_values = []
    B08_values = []
    B8A_values = []
    B11_values = []
    B12_values = []

# Iterate over the latitudes and longitudes, and extract the corresponding band values
    for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc="Mapping values"):
    # Assuming the correct dimensions are 'y' and 'x' (replace these with actual names from data.coords)
    
        B01_value = data.sel(x=lon, y=lat,  band=1, method="nearest").values
        B01_values.append(B01_value)
    
        B02_value = data.sel(x=lon, y=lat, band=2, method="nearest").values
        B02_values.append(B02_value)
        
        B03_value = data.sel(x=lon, y=lat, band=3, method="nearest").values
        B03_values.append(B03_value)
    
        B04_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
        B04_values.append(B04_value)
                
        B05_value = data.sel(x=lon, y=lat,  band=5, method="nearest").values
        B05_values.append(B05_value)
    
        B06_value = data.sel(x=lon, y=lat, band=6, method="nearest").values
        B06_values.append(B06_value)
        
        B07_value = data.sel(x=lon, y=lat, band=7, method="nearest").values
        B07_values.append(B07_value)
    
        B08_value = data.sel(x=lon, y=lat, band=8, method="nearest").values
        B08_values.append(B08_value)
        
        B8A_value = data.sel(x=lon, y=lat,  band=9, method="nearest").values
        B8A_values.append(B8A_value)
    
        B11_value = data.sel(x=lon, y=lat, band=10, method="nearest").values
        B11_values.append(B11_value)
        
        B12_value = data.sel(x=lon, y=lat, band=11, method="nearest").values
        B12_values.append(B12_value)
    
    # Create a DataFrame with the band values
    # Create a DataFrame to store the band values
    df['B01'] = B01_values
    df['B02'] = B02_values
    df['B03'] = B03_values
    df['B04'] = B04_values
    df['B05'] = B05_values
    df['B06'] = B06_values
    df['B07'] = B07_values
    df['B08'] = B08_values
    df['B8A'] = B8A_values
    df['B11'] = B11_values
    df['B12'] = B12_values
    
    return df

In [4]:
def get_data(df):
    bfa = get_building_footprint_area_data(df)
    bands = get_bands(df)

    merged_df = pd.merge(bfa, 
                         bands,
                         on=['Longitude', 'Latitude', 'UHI Index'])
    
    return(merged_df)

In [45]:
validation1 = get_data(pd.read_csv('Submission_template_UHI2025-v2.csv'))
validation = pd.merge(validation1,
                      pd.read_csv('validation_weather.csv'),
                      on=['Longitude', 'Latitude', 'UHI Index'])

/Users/alexyi/Desktop/school/vs_code/dat_eng_200/EY-Data-Chal/.venv/lib/python3.13/site-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
Mapping values: 100%|██████████| 1040/1040 [00:03<00:00, 276.85it/s]


In [46]:
train_data = pd.read_csv('light_building_weather_data.csv')
train_data = train_data.drop(columns='Unnamed: 0')

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
X = train_data.drop(columns=['Longitude', 
                            'Latitude', 
                            'datetime', 
                            'UHI Index',
                            'Area'])
y = train_data['UHI Index']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [48]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [500,600,800],
    'max_depth': [None, 50,75,100],
    'min_samples_leaf': [1],
    'max_features': ['sqrt']
}

# Initialize the model
rf = RandomForestRegressor()

# Setup GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')

# Fit to training data
grid_search.fit(X, y)

# Output the best parameters
print("Best Hyperparameters:", grid_search.best_params_)




Best Hyperparameters: {'max_depth': 50, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 500}


In [51]:
validation_test = validation.drop(columns=['Longitude', 'Latitude', 'UHI Index','datetime', 'Unnamed: 0'])
best_rf = grid_search.best_estimator_

In [52]:
final_predictions = best_rf.predict(validation_test)
final_predictions_series = pd.Series(final_predictions)



submission_df = pd.DataFrame({'Longitude':validation['Longitude'].values, \
                               'Latitude':validation['Latitude'].values, \
                                'UHI Index':final_predictions_series.values})


In [60]:
submission_df.to_csv("submission2.csv",index=False)

In [55]:
validation_df = pd.read_csv('Submission_template_UHI2025-v2.csv')

In [59]:
sum(validation_df['Latitude'] == submission_df['Latitude'])

1040

In [68]:
submission_df['Longitude'][0]

np.float64(-73.971665)

In [67]:
validation_df['Longitude'][0]

np.float64(-73.971665)

In [69]:
sub1 = pd.read_csv('67053f23bf0662f09fed2f3a-679bc36eb6a3410a9ebc6888-submission.csv')

In [71]:
sum(sub1['Longitude'] == submission_df['Longitude'])

1040